In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2022-ml-midterm-p1/submit.csv
/kaggle/input/2022-ml-midterm-p1/train.csv
/kaggle/input/2022-ml-midterm-p1/test.csv


In [2]:
import random
import os
seed = 42
random.seed(seed)
np.random.seed(seed)
os.environ["PYTHONHASHSEED"] = str(seed)

In [3]:
sample = pd.read_csv('../input/2022-ml-midterm-p1/submit.csv')
test = pd.read_csv('../input/2022-ml-midterm-p1/test.csv')
train = pd.read_csv('../input/2022-ml-midterm-p1/train.csv')

In [4]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 15 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   index     242 non-null    int64  
 1   age       242 non-null    int64  
 2   sex       242 non-null    int64  
 3   cp        242 non-null    int64  
 4   trestbps  242 non-null    int64  
 5   chol      242 non-null    int64  
 6   fbs       242 non-null    int64  
 7   restecg   242 non-null    int64  
 8   thalach   242 non-null    int64  
 9   exang     242 non-null    int64  
 10  oldpeak   242 non-null    float64
 11  slope     242 non-null    int64  
 12  ca        242 non-null    int64  
 13  thal      242 non-null    int64  
 14  target    242 non-null    int64  
dtypes: float64(1), int64(14)
memory usage: 28.5 KB


In [5]:
test.head()

,index,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,3,37,1,3,130,250,0,0,187,0,3.5,3,0,3
1,6,62,0,4,140,268,0,2,160,0,3.6,3,2,3
2,21,58,0,1,150,283,1,2,162,0,1.0,1,0,3
3,24,60,1,4,130,206,0,2,132,1,2.4,2,2,7
4,31,60,1,4,117,230,1,0,160,1,1.4,1,2,7


In [6]:
train.head()

,index,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0
4,5,56,1,2,120,236,0,0,178,0,0.8,1,0,3,0


In [7]:
train = train.drop(['index'], axis = 1)
test = test.drop(['index'], axis = 1)

In [8]:
trainX = train.drop(['target'], axis = 1)
trainY = train['target']

In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 242 entries, 0 to 241
Data columns (total 14 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       242 non-null    int64  
 1   sex       242 non-null    int64  
 2   cp        242 non-null    int64  
 3   trestbps  242 non-null    int64  
 4   chol      242 non-null    int64  
 5   fbs       242 non-null    int64  
 6   restecg   242 non-null    int64  
 7   thalach   242 non-null    int64  
 8   exang     242 non-null    int64  
 9   oldpeak   242 non-null    float64
 10  slope     242 non-null    int64  
 11  ca        242 non-null    int64  
 12  thal      242 non-null    int64  
 13  target    242 non-null    int64  
dtypes: float64(1), int64(13)
memory usage: 26.6 KB


In [10]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 61 entries, 0 to 60
Data columns (total 13 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       61 non-null     int64  
 1   sex       61 non-null     int64  
 2   cp        61 non-null     int64  
 3   trestbps  61 non-null     int64  
 4   chol      61 non-null     int64  
 5   fbs       61 non-null     int64  
 6   restecg   61 non-null     int64  
 7   thalach   61 non-null     int64  
 8   exang     61 non-null     int64  
 9   oldpeak   61 non-null     float64
 10  slope     61 non-null     int64  
 11  ca        61 non-null     int64  
 12  thal      61 non-null     int64  
dtypes: float64(1), int64(12)
memory usage: 6.3 KB


In [11]:
# 심장 질환 '여부'를 판단하는 것이므로 1이상이면 전부 '유'로 취급
tmp = np.asarray(trainY)
tmp[tmp > 0] = 1
#train.target[train.target > 0] = 1

In [12]:
trainY = tmp

In [13]:
# 데이터 값 정규화
from sklearn.preprocessing import RobustScaler
scaler = RobustScaler()
trainX_sc = scaler.fit_transform(trainX)
test_sc = scaler.transform(test)

In [14]:
# from sklearn.linear_model import LogisticRegression
# logis = LogisticRegression()
# logis.fit(trainX_sc, trainY)
# pred = logis.predict(test_sc)
# -> 0.80327

# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# lda = LinearDiscriminantAnalysis()
# trainX_sc_lda = lda.fit_transform(trainX_sc, trainY)
# test_sc_lda = lda.transform(test_sc)


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(trainX_sc, trainY, test_size=0.33, random_state=42)


from sklearn.ensemble import RandomForestClassifier
# 랜덤포레스트분류기 호출
clf = RandomForestClassifier(max_depth = 2, random_state=42)
# trainX_sc,trainY 로 학습시키기
clf.fit(trainX_sc, trainY)
print(clf.score(X_test, y_test))
# 모델로 test 예측하기
pred = clf.predict(test_sc)

# from sklearn.neighbors import KNeighborsClassifier
# neigh = KNeighborsClassifier(n_neighbors=10, weights = 'distance')
# neigh.fit(trainX_sc, trainY)
# pred = neigh.predict(test_sc)

0.9125


In [15]:
sample['target'] = pred


print(sample)
sample.to_csv('submission.csv', index = False)

    index  target
0       3       0
1       6       1
2      21       0
3      24       1
4      31       1
..    ...     ...
56    293       1
57    294       0
58    297       1
59    298       1
60    302       0

[61 rows x 2 columns]


In [16]:
sample.head()

,index,target
0,3,0
1,6,1
2,21,0
3,24,1
4,31,1


logistic regression, LDA+knn으로 예측한 성능보다 random forest로 예측한 성능이 더 좋게 나왔다.
데이터의 크기가 작아 random하게 forest를 생성하는 것이 우수한 성능을 보이는 걸로 분석했다.

# Review
마지막 마크다운 보면 마음이 급한 게 여기까지 느껴지는 코드..
실은 내가 왜 random forest를 선택했는지도 모르겠고
이렇게 푼 게 왜 리더보드 1등을 했는지도 모르겠다..
1. 심장 질환 '여부'를 판단하는 것이므로 1보다 크면 전부 '유'로 취급
➡️ 시험 보기 직전에 작년 시험문제 엄청 조금 훑었는데 이 부분을 잘 기억해둬서 좋은 성능을 낼 수 있었던 것 같다. '여부'에 focus !
2. 데이터의 범위가 0.xx ~ 286 정도로 광범위하기 때문에 RobustScaler를 사용하여 정규화를 시켜주었다.
3. RandomForestClassifier(max_depth = 2, random_state = 42)가 0.90163의 성능을 보였다. 
4. baseline은 KNeighborsClassifiter였다.
5. train_test_split 하지 않은 데이터로 학습시킨 모델의 성능이 더 우수했다.


⭐️ **A value is trying to be set on a copy of a slice from a DataFrame 
error
dataframe 상태에서 값을 변경하면 바뀐 dataframe을 원본 dataframe에 덮어씌우는 과정에서 시간이 많이 걸릴 수 있어서 warning.
➡️ numpy array로 변경 후 진행.**
